# Deferred After the inputs have been created

The input_df_dict dictionary contains the following information (keys)
 - billings
 - ADBE_cal
 - bookings
 - FX_forwards
 - FX_rates


In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pickle


In [2]:
input_df_dict = pickle.load(open('../data/processed/all_inputs.p', 'rb'))

df_billings = input_df_dict['billings']
df_ADBE_cal = input_df_dict['ADBE_cal']
df_bookings = input_df_dict['bookings']
df_FX_fwds  = input_df_dict['FX_forwards']
df_FX_rates = input_df_dict['FX_rates']

In [11]:
df_bookings.tail(20)

BU                    segment   product    geo  \
5994  Experience Cloud  Adobe Experience Manager     Forms   AMER    
5995  Experience Cloud  Adobe Experience Manager     Forms   AMER    
5996  Experience Cloud  Adobe Experience Manager     Forms   AMER    
5997  Experience Cloud  Adobe Experience Manager     Forms   AMER    
5998  Experience Cloud                   Magento   Magento   EMEA    
5999  Experience Cloud                   Magento   Magento   EMEA    
6000  Experience Cloud                   Magento   Magento   EMEA    
6001  Experience Cloud                   Magento   Magento   EMEA    
6002  Experience Cloud                   Magento   Magento   EMEA    
6003  Experience Cloud                   Magento   Magento   EMEA    
6004  Experience Cloud                   Magento   Magento   EMEA    
6005  Experience Cloud                   Magento   Magento   EMEA    
6006  Experience Cloud                   Magento   Magento   AMER    
6007  Experience Cloud                   Magento   Magento   AMER    
6008  Experience Cloud                   Magento   Magento   AMER    
6009  Experience Cloud                   Magento   Magento   AMER    
6010  Experience Cloud                   Magento   Magento   AMER    
6011  Experience Cloud                   Magento   Magento   AMER    
6012  Experience Cloud                   Magento   Magento   AMER    
6013  Experience Cloud                   Magento   Magento   AMER    

              country                  booking_type  Quarter     US_amount  \
5994   United States   Total Subscription Attrition  Q3 2020  4.923126e+01   
5995   United States   Total Subscription Attrition  Q3 2020 -4.923126e+01   
5996   United States   Total Subscription Attrition  Q4 2020  4.991613e+01   
5997   United States   Total Subscription Attrition  Q4 2020 -4.991613e+01   
5998         Germany                     Consulting  Q1 2020  1.542400e+06   
5999  United Kingdom                     Consulting  Q1 2020  3.856000e+05   
6000         Germany                     Consulting  Q2 2020  1.950400e+06   
6001  United Kingdom                     Consulting  Q2 2020  4.876000e+05   
6002         Germany                     Consulting  Q3 2020  1.926400e+06   
6003  United Kingdom                     Consulting  Q3 2020  4.816000e+05   
6004         Germany                     Consulting  Q4 2020  1.662400e+06   
6005  United Kingdom                     Consulting  Q4 2020  4.156000e+05   
6006   United States                     Consulting  Q1 2020  1.735200e+06   
6007   United States                     Consulting  Q1 2020  4.338000e+05   
6008   United States                     Consulting  Q2 2020  2.194200e+06   
6009   United States                     Consulting  Q2 2020  5.485500e+05   
6010   United States                     Consulting  Q3 2020  2.167200e+06   
6011   United States                     Consulting  Q3 2020  5.418000e+05   
6012   United States                     Consulting  Q4 2020  1.870200e+06   
6013   United States                     Consulting  Q4 2020  4.675500e+05   

              Country Currency  
5994   United States       USD  
5995   United States       USD  
5996   United States       USD  
5997   United States       USD  
5998         Germany       EUR  
5999  United Kingdom       GBP  
6000         Germany       EUR  
6001  United Kingdom       GBP  
6002         Germany       EUR  
6003  United Kingdom       GBP  
6004         Germany       EUR  
6005  United Kingdom       GBP  
6006   United States       USD  
6007   United States       USD  
6008   United States       USD  
6009   United States       USD  
6010   United States       USD  
6011   United States       USD  
6012   United States       USD  
6013   United States       USD

## We need to split the bookings information from quarters into periods
For each currency and BU, we will take the percentage of annual bookings for each period in a quarter.
This percentage will be what we use to split the bookings into periods


In [13]:
# find the last period in the billings index
last_period = '2020-03'


array(['2019-07', '2019-08', '2019-09', '2019-10', '2019-11', '2019-12',
       '2020-01', '2020-02', '2020-03', '2015-01', '2015-02', '2015-03',
       '2015-04', '2015-05', '2015-06', '2015-07', '2015-08', '2015-09',
       '2015-10', '2015-11', '2015-12', '2016-01', '2016-02', '2016-03',
       '2016-04', '2016-05', '2016-06', '2016-07', '2016-08', '2016-09',
       '2016-10', '2016-11', '2016-12', '2017-01', '2017-02', '2017-03',
       '2017-04', '2017-05', '2017-06', '2017-07', '2017-08', '2017-09',
       '2017-10', '2017-11', '2017-12', '2018-01', '2018-02', '2018-03',
       '2018-04', '2018-05', '2018-06', '2018-07', '2018-08', '2018-09',
       '2018-10', '2018-11', '2018-12', '2019-01', '2019-02', '2019-03',
       '2019-04', '2019-05', '2019-06'], dtype=object)

In [14]:
list_BUs = df_bookings['BU'].unique()
list_curr = df_bookings['Currency'].unique()

In [15]:
list_BUs

array(['Experience Cloud', 'Creative', 'Document Cloud'], dtype=object)

In [17]:
# creating dataframe of zeros

l_BU = []
l_curr = []
for BU in list_BUs:
    for curr in list_curr:
        l_BU.append(BU)
        l_curr.append(curr)
print(l_BU)
print(l_curr)
l_zero = np.arrange(len(l_BU))

['Experience Cloud', 'Experience Cloud', 'Experience Cloud', 'Experience Cloud', 'Experience Cloud', 'Creative', 'Creative', 'Creative', 'Creative', 'Creative', 'Document Cloud', 'Document Cloud', 'Document Cloud', 'Document Cloud', 'Document Cloud']
['AUD', 'EUR', 'USD', 'JPY', 'GBP', 'AUD', 'EUR', 'USD', 'JPY', 'GBP', 'AUD', 'EUR', 'USD', 'JPY', 'GBP']


In [16]:
list_curr

array(['AUD', 'EUR', 'USD', 'JPY', 'GBP'], dtype=object)

In [20]:
l_zero = np.zeros(len(l_BU))
l_zero

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [21]:
data= {'BU':l_BU, 'curr':l_curr, 
      'Q1':l_zero,
      'Q2':l_zero,
      'Q3':l_zero,
      'Q4':l_zero,
      'P01':l_zero,
      'P02':l_zero,
      'P03':l_zero,
      'P04':l_zero,
      'P05':l_zero,
      'P06':l_zero,
      'p07':l_zero,
       'P08':l_zero,
       'P09':l_zero,
       'P10':l_zero,
       'P11':l_zero,
       'P12':l_zero,
      }

df_book_period=pd.DataFrame(data)

In [22]:
df_book_period

BU curr   Q1   Q2   Q3   Q4  P01  P02  P03  P04  P05  P06  \
0   Experience Cloud  AUD  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
1   Experience Cloud  EUR  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
2   Experience Cloud  USD  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
3   Experience Cloud  JPY  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
4   Experience Cloud  GBP  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
5           Creative  AUD  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
6           Creative  EUR  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
7           Creative  USD  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
8           Creative  JPY  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
9           Creative  GBP  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
10    Document Cloud  AUD  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
11    Document Cloud  EUR  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
12    Document Cloud  USD  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
13    Document Cloud  JPY  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
14    Document Cloud  GBP  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   

    p07  P08  P09  P10  P11  P12  
0   0.0  0.0  0.0  0.0  0.0  0.0  
1   0.0  0.0  0.0  0.0  0.0  0.0  
2   0.0  0.0  0.0  0.0  0.0  0.0  
3   0.0  0.0  0.0  0.0  0.0  0.0  
4   0.0  0.0  0.0  0.0  0.0  0.0  
5   0.0  0.0  0.0  0.0  0.0  0.0  
6   0.0  0.0  0.0  0.0  0.0  0.0  
7   0.0  0.0  0.0  0.0  0.0  0.0  
8   0.0  0.0  0.0  0.0  0.0  0.0  
9   0.0  0.0  0.0  0.0  0.0  0.0  
10  0.0  0.0  0.0  0.0  0.0  0.0  
11  0.0  0.0  0.0  0.0  0.0  0.0  
12  0.0  0.0  0.0  0.0  0.0  0.0  
13  0.0  0.0  0.0  0.0  0.0  0.0  
14  0.0  0.0  0.0  0.0  0.0  0.0

In [23]:
df_bookings.head(10)

BU   segment   product    geo               country  \
0  Experience Cloud  Marketo   Marketo   ASIA   Aus and New Zealand    
1  Experience Cloud  Marketo   Marketo   ASIA   Aus and New Zealand    
2  Experience Cloud  Marketo   Marketo   ASIA   Aus and New Zealand    
3  Experience Cloud  Marketo   Marketo   ASIA   Aus and New Zealand    
4  Experience Cloud  Marketo   Marketo   ASIA   Aus and New Zealand    
5  Experience Cloud  Marketo   Marketo   ASIA   Aus and New Zealand    
6  Experience Cloud  Marketo   Marketo   ASIA   Aus and New Zealand    
7  Experience Cloud  Marketo   Marketo   ASIA   Aus and New Zealand    
8  Experience Cloud  Marketo   Marketo   ASIA   Aus and New Zealand    
9  Experience Cloud  Marketo   Marketo   ASIA   Aus and New Zealand    

                   booking_type  Quarter      US_amount               Country  \
0  Total Subscription Attrition  Q1 2020 -243336.527786  Aus and New Zealand    
1  Total Subscription Attrition  Q1 2020 -309237.732096  Aus and New Zealand    
2  Total Subscription Attrition  Q1 2020  -61767.378056  Aus and New Zealand    
3  Total Subscription Attrition  Q1 2020 -503969.121094  Aus and New Zealand    
4  Total Subscription Attrition  Q2 2020 -145084.185190  Aus and New Zealand    
5  Total Subscription Attrition  Q2 2020 -184376.364697  Aus and New Zealand    
6  Total Subscription Attrition  Q2 2020  -36827.474272  Aus and New Zealand    
7  Total Subscription Attrition  Q2 2020 -325745.459992  Aus and New Zealand    
8  Total Subscription Attrition  Q3 2020 -103805.125604  Aus and New Zealand    
9  Total Subscription Attrition  Q3 2020 -131917.973491  Aus and New Zealand    

  Currency  
0      AUD  
1      AUD  
2      AUD  
3      AUD  
4      AUD  
5      AUD  
6      AUD  
7      AUD  
8      AUD  
9      AUD

In [48]:
# fill in the quarters
for i in range(len(df_book_period['BU'])):
    
    this_BU = df_book_period['BU'][i]
    this_curr = df_book_period['curr'][i]
    this_slice = df_bookings[(df_bookings['BU']==this_BU)&
                          (df_bookings['Currency']==this_curr)]
    
    this_Q1= this_slice[this_slice['Quarter']=='Q1 2020']
    sum_Q1 = this_Q1['US_amount'].sum()
    df_book_period['Q1'].loc[i]=sum_Q1
    
    this_Q2= this_slice[this_slice['Quarter']=='Q2 2020']
    sum_Q2 = this_Q2['US_amount'].sum()
    df_book_period['Q2'].loc[i]=sum_Q2
    
    this_Q3= this_slice[this_slice['Quarter']=='Q4 2020']
    sum_Q3 = this_Q3['US_amount'].sum()
    df_book_period['Q3'].loc[i]=sum_Q3
    
    this_Q4= this_slice[this_slice['Quarter']=='Q4 2020']
    sum_Q4 = this_Q4['US_amount'].sum()
    df_book_period['Q4'].loc[i]=sum_Q4
    
    

0   Experience Cloud   AUD
1   Experience Cloud   EUR
2   Experience Cloud   USD
3   Experience Cloud   JPY
4   Experience Cloud   GBP
5   Creative   AUD
6   Creative   EUR
7   Creative   USD
8   Creative   JPY
9   Creative   GBP
10   Document Cloud   AUD
11   Document Cloud   EUR
12   Document Cloud   USD
13   Document Cloud   JPY
14   Document Cloud   GBP


/Users/davidsmith/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [49]:
df_book_period

BU curr            Q1            Q2            Q3  \
0   Experience Cloud  AUD  7.433434e+06  1.358439e+07  1.816287e+07   
1   Experience Cloud  EUR  3.499845e+07  3.721689e+07  7.695415e+07   
2   Experience Cloud  USD  1.247171e+08  1.986764e+08  3.135637e+08   
3   Experience Cloud  JPY  1.182163e+07  2.024655e+07  2.681428e+07   
4   Experience Cloud  GBP  2.157488e+07  2.645604e+07  4.759333e+07   
5           Creative  AUD  1.393692e+07  1.379181e+07  2.364523e+07   
6           Creative  EUR  0.000000e+00  0.000000e+00  0.000000e+00   
7           Creative  USD  2.075290e+07  1.613099e+07  3.783942e+07   
8           Creative  JPY  2.711939e+06  6.155972e+06  5.318174e+06   
9           Creative  GBP  1.579663e+07  1.104858e+07  2.342099e+07   
10    Document Cloud  AUD  2.705300e+06  2.335687e+06  3.061921e+06   
11    Document Cloud  EUR  0.000000e+00  0.000000e+00  0.000000e+00   
12    Document Cloud  USD  1.282261e+07  1.218840e+07  2.995207e+07   
13    Document Cloud  JPY  5.782041e+05  9.107898e+05  1.260247e+06   
14    Document Cloud  GBP  7.157459e+06  7.359043e+06  1.397187e+07   

              Q4  P01  P02  P03  P04  P05  P06  p07  P08  P09  P10  P11  P12  
0   1.816287e+07  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
1   7.695415e+07  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
2   3.135637e+08  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
3   2.681428e+07  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
4   4.759333e+07  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
5   2.364523e+07  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
6   0.000000e+00  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
7   3.783942e+07  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
8   5.318174e+06  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
9   2.342099e+07  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
10  3.061921e+06  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
11  0.000000e+00  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
12  2.995207e+07  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
13  1.260247e+06  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
14  1.397187e+07  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

In [57]:
# list of quarters for the percentages
list_q2 = ['2019-04', '2019-05', '2019-06']
list_q3 = ['2019-07', '2019-08', '2019-09']
list_q4 = ['2019-10', '2019-11', '2019-12']
list_q1 = [ '2020-01', '2020-02', '2020-03']

list_periods = ['2019-04', '2019-05', '2019-06',
                '2019-07', '2019-08', '2019-09',
                '2019-10', '2019-11', '2019-12',
                '2020-01', '2020-02', '2020-03']
print(list_periods)
list_p_headers = ['P01', 'P02', 'P03',
                  'P04', 'P05', 'P06',
                  'P07', 'P08', 'P09',
                  'P10', 'P11', 'P12'
                 ]
print(list_p_headers)

['2019-04', '2019-05', '2019-06', '2019-07', '2019-08', '2019-09', '2019-10', '2019-11', '2019-12', '2020-01', '2020-02', '2020-03']
['P01', 'P02', 'P03', 'P04', 'P05', 'P06', 'P07', 'P08', 'P09', 'P10', 'P11', 'P12']


In [ ]:
# fill in the periods
for i in range(len(df_book_period['BU'])):
    
    this_BU = df_book_period['BU'][i]
    this_curr = df_book_period['curr'][i]
    this_period = 
    this_slice = df_billings[(df_billings['BU']==this_BU)&
                          (df_billings['curr']==this_curr)]
    
    this_P1= this_slice[this_slice['period']=='Q1 2020']
    
    sum_Q1 = this_Q1['US_amount'].sum()
    df_book_period['Q1'].loc[i]=sum_Q1
    
    this_Q2= this_slice[this_slice['Quarter']=='Q2 2020']
    sum_Q2 = this_Q2['US_amount'].sum()
    df_book_period['Q2'].loc[i]=sum_Q2
    
    this_Q3= this_slice[this_slice['Quarter']=='Q4 2020']
    sum_Q3 = this_Q3['US_amount'].sum()
    df_book_period['Q3'].loc[i]=sum_Q3
    
    this_Q4= this_slice[this_slice['Quarter']=='Q4 2020']
    sum_Q4 = this_Q4['US_amount'].sum()
    df_book_period['Q4'].loc[i]=sum_Q4

In [51]:
df_billings

curr                BU   period  recognized_DC  recognized_US  \
0     ARS          Creative  2019-07        6786.00         155.12   
1     ARS          Creative  2019-08       16472.00         390.02   
2     ARS          Creative  2019-09       19205.00         405.51   
3     ARS          Creative  2019-10       30382.19         532.13   
4     ARS          Creative  2019-11       59049.00        1023.67   
...   ...               ...      ...            ...            ...   
4178  JPY          DX Other  2018-04           0.00           0.00   
4179  JPY          DX Other  2018-09           0.00           0.00   
4180  JPY          DX Other  2018-11           0.00           0.00   
4181  MXP  Experience Cloud  2017-06           0.00           0.00   
4182  NOK   Other Solutions  2015-10           0.00           0.00   

      service_DC  service_US  deferred_B_DC  deferred_B_US      _merge  ...  \
0            0.0         0.0            0.0            0.0   left_only  ...   
1            0.0         0.0            0.0            0.0   left_only  ...   
2            0.0         0.0            0.0            0.0   left_only  ...   
3            0.0         0.0            0.0            0.0   left_only  ...   
4            0.0         0.0            0.0            0.0   left_only  ...   
...          ...         ...            ...            ...         ...  ...   
4178         0.0         0.0            0.0            0.0  right_only  ...   
4179         0.0         0.0            0.0            0.0  right_only  ...   
4180         0.0         0.0            0.0            0.0  right_only  ...   
4181         0.0         0.0            0.0            0.0  right_only  ...   
4182         0.0         0.0            0.0            0.0  right_only  ...   

      deferred_6M_DC  deferred_1Y_DC  deferred_2Y_DC  deferred_3Y_DC  \
0                0.0       953875.89             0.0             0.0   
1                0.0       963180.00             0.0             0.0   
2                0.0      2263704.00             0.0             0.0   
3                0.0      1318346.81             0.0             0.0   
4                0.0      1494162.20             0.0             0.0   
...              ...             ...             ...             ...   
4178             0.0    -11142000.00             0.0             0.0   
4179             0.0            0.00             0.0             0.0   
4180             0.0            0.00             0.0             0.0   
4181         84000.0            0.00             0.0             0.0   
4182             0.0         -202.50             0.0             0.0   

      deferred_1M_US  deferred_3M_US  deferred_6M_US  deferred_1Y_US  \
0       1.460911e+05             0.0             0.0        21566.14   
1       1.732747e+05             0.0             0.0        22781.91   
2       2.176112e+05             0.0             0.0        46297.25   
3       1.817068e+05             0.0             0.0        23376.52   
4       1.960320e+05             0.0             0.0        25782.35   
...              ...             ...             ...             ...   
4178    0.000000e+00             0.0             0.0    -11142000.00   
4179    1.025000e+07             0.0             0.0            0.00   
4180    1.148333e+06             0.0             0.0            0.00   
4181    0.000000e+00             0.0         84000.0            0.00   
4182    0.000000e+00             0.0             0.0         -202.50   

      deferred_2Y_US  deferred_3Y_US  
0                0.0             0.0  
1                0.0             0.0  
2                0.0             0.0  
3                0.0             0.0  
4                0.0             0.0  
...              ...             ...  
4178             0.0             0.0  
4179             0.0             0.0  
4180             0.0             0.0  
4181             0.0             0.0  
4182             0.0             0.0  

[4096 rows x 22 

[['2020-01', '2020-02', '2020-03'],
 ['2019-04', '2019-05', '2019-06'],
 ['2019-07', '2019-08', '2019-09'],
 ['2019-10', '2019-11', '2019-12']]